# 🎉 Simple Network Simulation

---
## 🔍 Notebook objectives

This notebook contains simulation of 2 excitatory regular spiking (RS) neuron and 2 inhibitory fast spiking (FS) interneurons with symmetric weights composition but with different input current (to observe competition dynamics).

# 🎒 Setup

## ⬇️ Imports

In [2]:
from utils.simple_network_simulation import network_simulation_run
from utils.plots import *

import numpy as np
import yaml #reading env consts

## 🛠️ Simulation Utils

In [3]:
#RS neuron parameters
with open('utils/RS.yaml', 'r', encoding="utf-8") as fin:
    params_RS = yaml.load(fin, Loader=yaml.FullLoader)

#FS neuron parameters
with open('utils/FS.yaml', 'r', encoding="utf-8") as fin:
    params_FS = yaml.load(fin, Loader=yaml.FullLoader)

#receptor kinetics parameters
with open('utils/receptor_kinetics.yaml', 'r', encoding="utf-8") as fin:
    params_receptor_kinetics = yaml.load(fin, Loader=yaml.FullLoader)

In [4]:
#simulation time
t_min = 0
t_max = 1000 #in ms -> 1(s) of simulation
sim_steps = 10000

T = np.linspace(t_min, t_max, sim_steps)
delta_T = t_max/sim_steps

# 🧪 Experiments

## 1️⃣ Competition

In [5]:
#2 RS and 2 FS
Ne = 2
Ni = 2

weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j
        
weights[0, 2] = 5
weights[1, 3] = 5
weights[2, 1] = 10
weights[3, 0] = 10

currents = np.array(np.zeros((len(T), Ne + Ni)))
currents[:, 0] = 1250
currents[:, 1] = 1200

In [6]:
start_state = np.column_stack((np.append(params_RS["v"]*np.ones(Ne), params_FS["v"]*np.ones(Ni)), np.append(params_RS["u"]*np.ones(Ne), params_FS["u"]*np.ones(Ni)))) #matrix (Ne + Ni) x 2 - (v, u) for each neuron

params_network = {"a": np.append(params_RS["a"]*np.ones(Ne), params_FS["a"]*np.ones(Ni)), 
          "b": np.append(params_RS["b"]*np.ones(Ne), params_FS["b"]*np.ones(Ni)), 
          "c": np.append(params_RS["c"]*np.ones(Ne), params_FS["c"]*np.ones(Ni)), 
          "d": np.append(params_RS["d"]*np.ones(Ne), params_FS["d"]*np.ones(Ni)), 
          "C": np.append(params_RS["C"]*np.ones(Ne), params_FS["C"]*np.ones(Ni)), 
          "k": np.append(params_RS["k"]*np.ones(Ne), params_FS["k"]*np.ones(Ni)),
          "v_peak": np.append(params_RS["v_peak"]*np.ones(Ne), params_FS["v_peak"]*np.ones(Ni)), 
          "v_r": np.append(params_RS["v_r"]*np.ones(Ne), params_FS["v_r"]*np.ones(Ni)), 
          "v_t": np.append(params_RS["v_t"]*np.ones(Ne), params_FS["v_t"]*np.ones(Ni)),
          "tau_ampa": params_receptor_kinetics["tau_ampa"]*np.ones((Ne + Ni, Ne + Ni)),
          "tau_nmda": params_receptor_kinetics["tau_nmda"]*np.ones((Ne + Ni, Ne + Ni)),
          "tau_gabaa": params_receptor_kinetics["tau_gabaa"]*np.ones((Ne + Ni, Ne + Ni)),
          "tau_gabab": params_receptor_kinetics["tau_gabab"]*np.ones((Ne + Ni, Ne + Ni))}

In [10]:
states, firings, conductances = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, currents, params_network)

In [11]:
_, fig = plot_all_membrane_voltage(states[:, :, 0], T)
fig.show()

In [12]:
_, fig = firing_rates(firings, T)
fig.show()

In [13]:
np.sum(firings, axis = 0)

array([30.,  2., 60.,  2.])